# Segmenting and Clustering Neighborhood in Toronto

In [1]:
import pandas as pd
import numpy as np
import requests
import bs4
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import folium

In [2]:
## getting data from web
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = bs4.BeautifulSoup(res.text, 'html.parser')

### first part of assignment

### html table that contains required data

In [3]:
table = soup.find('table', class_ = 'wikitable sortable')
table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Community
</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<

In [4]:
data = []
for body in table.findAll('tbody'):
    for rows in body.findAll('tr'):
        for col in rows.findAll('td'):
            data.append(col.text)
            

### Cleaned data

In [5]:
un_wanted = ['\n']
for i in range(len(data)):
    for j in un_wanted:
        data[i] = data[i].replace(j, '')
data

['M1A',
 'Not assigned',
 'Not assigned',
 'M2A',
 'Not assigned',
 'Not assigned',
 'M3A',
 'North York',
 'Parkwoods',
 'M4A',
 'North York',
 'Victoria Village',
 'M5A',
 'Downtown Toronto',
 'Regent Park, Harbourfront',
 'M6A',
 'North York',
 'Lawrence Manor, Lawrence Heights',
 'M7A',
 'Downtown Toronto',
 "Queen's Park, Ontario Provincial Government",
 'M8A',
 'Not assigned',
 'Not assigned',
 'M9A',
 'Etobicoke',
 'Islington Avenue, Humber Valley Village',
 'M1B',
 'Scarborough',
 'Malvern, Rouge',
 'M2B',
 'Not assigned',
 'Not assigned',
 'M3B',
 'North York',
 'Don Mills',
 'M4B',
 'East York',
 'Parkview Hill, Woodbine Gardens',
 'M5B',
 'Downtown Toronto',
 'Garden District, Ryerson',
 'M6B',
 'North York',
 'Glencairn',
 'M7B',
 'Not assigned',
 'Not assigned',
 'M8B',
 'Not assigned',
 'Not assigned',
 'M9B',
 'Etobicoke',
 'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale',
 'M1C',
 'Scarborough',
 'Rouge Hill, Port Union, Highland Creek',
 'M2C',


In [6]:
PostalCode = []
Borough = []
Neighborhood = []

for i in range(0, len(data), 3):
    PostalCode.append(data[i])

for i in range(1, len(data), 3):
    Borough.append(data[i])

for i in range(2, len(data), 3):
    Neighborhood.append(data[i])

df = pd.DataFrame(list(zip(PostalCode, Borough, Neighborhood)), columns = ['PostalCode', 'Borough', 'Neighborhood'])
df = df[df['Borough'] != 'Not assigned'].reset_index(drop=True)

In [7]:
df.head(15)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [8]:
df.shape

(103, 3)

### Second part of assignment starts from here

In [9]:
ll_df = pd.read_csv('Geospatial_Coordinates.csv')

ll_df.rename(columns={'Postal Code': 'PostalCode'}, inplace = True)
ll_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
df_final = pd.merge(df, ll_df, on='PostalCode', how='inner')
df_final.head(15)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [11]:
df_final.shape

(103, 5)

### Third part starts here

In [12]:
from pandas.io.json import json_normalize

In [13]:
df_final['Borough'].unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

In [14]:
# lets copy the data in another data frame so that we would not lose any data in case we need later
df_test = df_final

### We will analyze those borough which contains Toronto in there name

In [15]:
# Select Borough which have Toronto in there name
# There are four such borough
borough_toronto = ['North York', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'Mississauga']
for i in borough_toronto:
    df_test = df_test[df_test['Borough'] != i].reset_index(drop=True)

In [16]:
df_test

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [17]:
toronto_neighborhood_data = df_test

### Ploting neighborhood on map

In [18]:
toronto_neighborhood_map = folium.Map(location=[43.6532, -79.3832],
                                     zoom_start=10)

for lat, long, neigh in zip(toronto_neighborhood_data['Latitude'], toronto_neighborhood_data['Longitude'], toronto_neighborhood_data['Neighborhood']):
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=folium.Popup(str(neigh), parse_html=True),
        color='Blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_neighborhood_map)

toronto_neighborhood_map

#### Clustering the neighborhoods

In [19]:
YOUR_CLIENT_ID = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
CLIENT_ID = YOUR_CLIENT_ID # your Foursquare ID
YOUR_CLIENT_SECRET = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
CLIENT_SECRET = YOUR_CLIENT_SECRET # your Foursquare Secret
VERSION = '20200715' # Foursquare API version
LIMIT = 30

#### function to get venues in the neighborhoods(using foursquare-api)

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}
        results = requests.get(url, headers=headers).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [21]:
toronto_venues = getNearbyVenues(names=df_test['Neighborhood'], latitudes=df_test['Latitude'], longitudes=df_test['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


In [22]:
toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.654260,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
...,...,...,...,...,...,...,...
848,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,TTC Russell Division,43.664908,-79.322560,Light Rail Station
849,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Jonathan Ashbridge Park,43.664702,-79.319898,Park
850,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Olliffe On Queen,43.664503,-79.324768,Butcher
851,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,ONE Academy,43.662253,-79.326911,Gym / Fitness Center


In [23]:
## different categories of venues present in toronto
len(toronto_venues['Venue Category'].unique())

192

In [24]:
## assigning dummies for differnt venues
toronto_onehot = pd.get_dummies(toronto_venues['Venue Category'])

## drop neighborhood column(already existing) and add new neighborhood same as in toronto_venues
toronto_onehot.drop(['Neighborhood'], axis=1, inplace=True)
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

## bring neighborhood as column 1 (because it would be easy to analyze)
columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])          ## list of columns as required
toronto_onehot = toronto_onehot[columns]

toronto_onehot

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
848,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
849,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
850,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
851,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
## toronto_grouped gives the frequency of different venues in each neighborhood
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0625,0.0625,0.0625,0.125,0.125,0.0625,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333
5,Christie,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.00,0.000000,...,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Commerce Court, Victoria Hotel",0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.066667,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Davisville,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### lets get top 8 most common venues in each neighborhood

In [26]:
## lets name the columns
columns = ['Neighborhood', '1st most commom', '2nd most common', '3rd most common', '4th most common', 
           '5th most common', '6th most common', '7th most common', '8th most common']

toronto_neighborhood_sorted = pd.DataFrame(columns=columns)
toronto_neighborhood_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

toronto_neighborhood_sorted.head()

,Neighborhood,1st most commom,2nd most common,3rd most common,4th most common,5th most common,6th most common,7th most common,8th most common
0,Berczy Park,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Brockton, Parkdale Village, Exhibition Place",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Business reply mail Processing Centre, South C...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"CN Tower, King and Spadina, Railway Lands, Har...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Central Bay Street,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
## function for shorting venues by frequency
## returns only a given number(n) of top venues(by frequency)

def venues_sorted_by_freq(venue_list, n):
    temp = venue_list.iloc[1:]
    top_venue_sorted = temp.sort_values(ascending=False)
    return top_venue_sorted.index.values[0:n]

In [28]:
for i in np.arange(toronto_grouped.shape[0]):
    toronto_neighborhood_sorted.iloc[i, 1:] = venues_sorted_by_freq(toronto_grouped.iloc[i, :], 8)

toronto_neighborhood_sorted.head(10)

,Neighborhood,1st most commom,2nd most common,3rd most common,4th most common,5th most common,6th most common,7th most common,8th most common
0,Berczy Park,Seafood Restaurant,Coffee Shop,Cocktail Bar,Beer Bar,Farmers Market,Japanese Restaurant,Basketball Stadium,Bistro
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Nightclub,Coffee Shop,Pet Store,Bakery,Intersection,Italian Restaurant
2,"Business reply mail Processing Centre, South C...",Auto Workshop,Pizza Place,Skate Park,Light Rail Station,Restaurant,Burrito Place,Recording Studio,Butcher
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport,Harbor / Marina,Coffee Shop,Plane,Rental Car Location,Sculpture Garden
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Yoga Studio,Spa,Middle Eastern Restaurant,Comic Shop,Park
5,Christie,Grocery Store,Café,Park,Athletics & Sports,Nightclub,Coffee Shop,Italian Restaurant,Restaurant
6,Church and Wellesley,Coffee Shop,Ethiopian Restaurant,Juice Bar,Escape Room,Hobby Shop,Café,Beer Bar,Ice Cream Shop
7,"Commerce Court, Victoria Hotel",Café,Restaurant,Japanese Restaurant,Gastropub,Coffee Shop,American Restaurant,Hotel,Gym / Fitness Center
8,Davisville,Dessert Shop,Coffee Shop,Gym,Sandwich Place,Italian Restaurant,Café,Pizza Place,Sushi Restaurant
9,Davisville North,Dance Studio,Gym / Fitness Center,Park,Breakfast Spot,Sandwich Place,Department Store,Food & Drink Shop,Hotel


### Cluster neighborhood using KMeans

In [29]:
n = 6 ## number of cluster
clustering_data = toronto_grouped.drop('Neighborhood', axis=1)

kmeans = KMeans(n_clusters=n,
               random_state=0)
kmeans.fit(clustering_data)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=6, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

In [30]:
## adding clustering labels
toronto_neighborhood_sorted.insert(0, 'Cluster lables', kmeans.labels_)

In [31]:
toronto_merged_data = toronto_neighborhood_data

toronto_merged_data = toronto_merged_data.join(toronto_neighborhood_sorted.set_index('Neighborhood'), on='Neighborhood')

In [32]:
toronto_merged_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster lables,1st most commom,2nd most common,3rd most common,4th most common,5th most common,6th most common,7th most common,8th most common
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Theater,Distribution Center,Dessert Shop
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2,Coffee Shop,Yoga Studio,College Auditorium,Smoothie Shop,Beer Bar,Sandwich Place,Portuguese Restaurant,Café
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Café,Coffee Shop,Clothing Store,Theater,Fast Food Restaurant,Bookstore,Japanese Restaurant,Diner
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Gastropub,Café,Japanese Restaurant,Coffee Shop,Restaurant,Park,Middle Eastern Restaurant,Hotel
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Pub,Health Food Store,Yoga Studio,Cuban Restaurant,Donut Shop,Dog Run,Distribution Center
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,Seafood Restaurant,Coffee Shop,Cocktail Bar,Beer Bar,Farmers Market,Japanese Restaurant,Basketball Stadium,Bistro
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,2,Coffee Shop,Café,Italian Restaurant,Yoga Studio,Spa,Middle Eastern Restaurant,Comic Shop,Park
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564,2,Grocery Store,Café,Park,Athletics & Sports,Nightclub,Coffee Shop,Italian Restaurant,Restaurant
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,2,Café,Steakhouse,Coffee Shop,Asian Restaurant,Restaurant,Plaza,Seafood Restaurant,Bar
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,2,Bakery,Pharmacy,Music Venue,Supermarket,Bank,Bar,Middle Eastern Restaurant,Gym / Fitness Center


### Ploting clusters on map

In [33]:
## latitide and longitude of toronto
lat_t, long_t = 43.6532, -79.3832

In [34]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

In [35]:
toronto_map = folium.Map(location=[lat_t, long_t], zome_start=10)

x = np.arange(n)
ys = [i + x + (i*x)**2 for i in range(n)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

for lat, long, neigh, lab in zip(toronto_merged_data['Latitude'], toronto_merged_data['Longitude'], toronto_merged_data['Neighborhood'], toronto_merged_data['Cluster lables']):
    folium.CircleMarker(
    [lat, long],
    radius=5,
    popup=folium.Popup(str(neigh)+' cluster: '+str(lab), parse_html=True),
    color=rainbow[lab-1],
    fill=True,
    fill_color=rainbow[lab-1],
    fill_opacity=0.7).add_to(toronto_map)

In [36]:
toronto_map

### Different Clusters

In [37]:
## cluster 1
toronto_merged_data.loc[toronto_merged_data['Cluster lables']==0, toronto_merged_data.columns[[1] + list(range(5, toronto_merged_data.shape[1]))]]

,Borough,Cluster lables,1st most commom,2nd most common,3rd most common,4th most common,5th most common,6th most common,7th most common,8th most common
4,East Toronto,0,Trail,Pub,Health Food Store,Yoga Studio,Cuban Restaurant,Donut Shop,Dog Run,Distribution Center
29,Central Toronto,0,Playground,Trail,Yoga Studio,Cuban Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store


In [38]:
## cluster 2
toronto_merged_data.loc[toronto_merged_data['Cluster lables']==1, toronto_merged_data.columns[[1] + list(range(5, toronto_merged_data.shape[1]))]]

,Borough,Cluster lables,1st most commom,2nd most common,3rd most common,4th most common,5th most common,6th most common,7th most common,8th most common
10,Downtown Toronto,1,Plaza,Café,Hotel,Park,Basketball Stadium,Dance Studio,Lake,Sporting Goods Shop
12,East Toronto,1,Greek Restaurant,Italian Restaurant,Bookstore,Ice Cream Shop,Yoga Studio,Pizza Place,Juice Bar,Restaurant
15,East Toronto,1,Park,Coffee Shop,Fish & Chips Shop,Brewery,Restaurant,Movie Theater,Italian Restaurant,Burrito Place
20,Central Toronto,1,Dance Studio,Gym / Fitness Center,Park,Breakfast Spot,Sandwich Place,Department Store,Food & Drink Shop,Hotel
22,West Toronto,1,Mexican Restaurant,Café,Bar,Thai Restaurant,Flea Market,Diner,Bookstore,Italian Restaurant
24,Central Toronto,1,Sandwich Place,Café,Coffee Shop,Park,History Museum,Donut Shop,Burger Joint,Liquor Store
25,West Toronto,1,Gift Shop,Breakfast Spot,Bar,Italian Restaurant,Eastern European Restaurant,Dog Run,Dessert Shop,Cuban Restaurant
26,Central Toronto,1,Dessert Shop,Coffee Shop,Gym,Sandwich Place,Italian Restaurant,Café,Pizza Place,Sushi Restaurant
28,West Toronto,1,Café,Pizza Place,Sushi Restaurant,Coffee Shop,Italian Restaurant,Indie Movie Theater,Bank,Bar
30,Downtown Toronto,1,Café,Vietnamese Restaurant,Mexican Restaurant,Cocktail Bar,Pizza Place,Record Shop,Dessert Shop,Dumpling Restaurant


In [39]:
## cluster 3
toronto_merged_data.loc[toronto_merged_data['Cluster lables']==2, toronto_merged_data.columns[[1] + list(range(5, toronto_merged_data.shape[1]))]]

,Borough,Cluster lables,1st most commom,2nd most common,3rd most common,4th most common,5th most common,6th most common,7th most common,8th most common
0,Downtown Toronto,2,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Theater,Distribution Center,Dessert Shop
1,Downtown Toronto,2,Coffee Shop,Yoga Studio,College Auditorium,Smoothie Shop,Beer Bar,Sandwich Place,Portuguese Restaurant,Café
2,Downtown Toronto,2,Café,Coffee Shop,Clothing Store,Theater,Fast Food Restaurant,Bookstore,Japanese Restaurant,Diner
3,Downtown Toronto,2,Gastropub,Café,Japanese Restaurant,Coffee Shop,Restaurant,Park,Middle Eastern Restaurant,Hotel
5,Downtown Toronto,2,Seafood Restaurant,Coffee Shop,Cocktail Bar,Beer Bar,Farmers Market,Japanese Restaurant,Basketball Stadium,Bistro
6,Downtown Toronto,2,Coffee Shop,Café,Italian Restaurant,Yoga Studio,Spa,Middle Eastern Restaurant,Comic Shop,Park
7,Downtown Toronto,2,Grocery Store,Café,Park,Athletics & Sports,Nightclub,Coffee Shop,Italian Restaurant,Restaurant
8,Downtown Toronto,2,Café,Steakhouse,Coffee Shop,Asian Restaurant,Restaurant,Plaza,Seafood Restaurant,Bar
9,West Toronto,2,Bakery,Pharmacy,Music Venue,Supermarket,Bank,Bar,Middle Eastern Restaurant,Gym / Fitness Center
11,West Toronto,2,Bar,Vietnamese Restaurant,Asian Restaurant,Coffee Shop,Malay Restaurant,Beer Store,Brewery,Record Shop


In [40]:
## cluster 4
toronto_merged_data.loc[toronto_merged_data['Cluster lables']==3, toronto_merged_data.columns[[1] + list(range(5, toronto_merged_data.shape[1]))]]

,Borough,Cluster lables,1st most commom,2nd most common,3rd most common,4th most common,5th most common,6th most common,7th most common,8th most common
19,Central Toronto,3,Garden,Music Venue,Yoga Studio,Cuban Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store


In [41]:
## cluster 5
toronto_merged_data.loc[toronto_merged_data['Cluster lables']==4, toronto_merged_data.columns[[1] + list(range(5, toronto_merged_data.shape[1]))]]

,Borough,Cluster lables,1st most commom,2nd most common,3rd most common,4th most common,5th most common,6th most common,7th most common,8th most common
18,Central Toronto,4,Park,Bus Line,Swim School,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner


In [42]:
## cluster 6
toronto_merged_data.loc[toronto_merged_data['Cluster lables']==5, toronto_merged_data.columns[[1] + list(range(5, toronto_merged_data.shape[1]))]]

,Borough,Cluster lables,1st most commom,2nd most common,3rd most common,4th most common,5th most common,6th most common,7th most common,8th most common
21,Central Toronto,5,Jewelry Store,Park,Trail,Sushi Restaurant,Dance Studio,Donut Shop,Dog Run,Distribution Center
33,Downtown Toronto,5,Park,Playground,Trail,Cuban Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store
